In [1]:
import json
import hashlib
from datetime import datetime
from typing import Any, Dict, Optional, Callable
import vertexai
from vertexai import agent_engines as _agent_engines
from google.cloud import storage

In [2]:
def get_or_create_agent_engine(display_name: str) :
    """
    Pobiera istniejący Agent Engine po nazwie wyświetlanej lub tworzy nowy, jeśli nie istnieje.
    """
    # 1. Pobierz listę wszystkich istniejących silników w projekcie
    all_engines = _agent_engines.list()
    
    # 2. Sprawdź, czy któryś z nich ma pasującą nazwę
    for engine in all_engines:
        if engine.display_name == display_name:
            print(f"INFO: Znaleziono i połączono z istniejącym Agent Engine: '{display_name}'")
            return engine
            
    # 3. Jeśli pętla się zakończyła i nic nie znaleziono, stwórz nowy silnik
    print(f"INFO: Nie znaleziono Agent Engine o nazwie '{display_name}'. Tworzenie nowego...")
    try:
        new_engine = agent_engines.create(
            display_name=display_name
        )
        print(f"INFO: Pomyślnie utworzono nowy Agent Engine.")
        return new_engine
    except Exception as e:
        print(f"KRYTYCZNY BŁĄD: Nie można utworzyć Agent Engine. Sprawdź konfigurację i uprawnienia. Błąd: {e}")
        exit()

In [3]:
MEMORY_ENGINE_DISPLAY_NAME="test_for_moa_debate"

In [4]:
agent_engine =get_or_create_agent_engine(MEMORY_ENGINE_DISPLAY_NAME)
AGENT_ENGINE_NAME = agent_engine.resource_name
print(AGENT_ENGINE_NAME)

INFO: Znaleziono i połączono z istniejącym Agent Engine: 'test_for_moa_debate'
projects/815755318672/locations/us-central1/reasoningEngines/6370486808450433024


In [5]:
from vertexai import Client  # przygotuj klienta wcześniej
client = Client(project="dark-data-discovery", location="us-central1")
from persist_missions_memory import persist_missions_to_vertex_memory

In [6]:
from persist_missions_memory_semantic_incremental import run_persistent_memory_ingestion

In [7]:
# --- 1. KONFIGURACJA ---
# Uzupełnij poniższe zmienne. Muszą być identyczne jak w skrypcie do odczytu.
PROJECT_ID = "dark-data-discovery"
LOCATION = "us-central1"
GCS_BUCKET = "memory_rag_for_agents" # Nazwa Twojego bucketa w Cloud Storage

# WAŻNE: Wklej PEŁNĄ NAZWĘ ZASOBU (Resource Name) swojego silnika agenta.
ENGINE_NAME = "projects/815755318672/locations/us-central1/reasoningEngines/6370486808450433024"

# Ścieżka do pliku JSON z danymi do wczytania
JSON_SOURCE_FILE = 'memory/learned_strategies.json'

In [9]:
from vertexai import Client  # przygotuj klienta wcześniej
client = Client(project="dark-data-discovery", location="us-central1")


gcs_client = storage.Client(project=PROJECT_ID)

print("✅ Klienci Google Cloud zainicjalizowani pomyślnie.")

try:
    # Uruchomienie głównej funkcji
    run_persistent_memory_ingestion(
        json_path=JSON_SOURCE_FILE,
        engine_name=ENGINE_NAME,
        vertex_client=client,
        gcs_client=gcs_client,
        gcs_bucket=GCS_BUCKET
    )
except Exception as e:
    print(f"\n❌ BŁĄD KRYTYCZNY PRZED URUCHOMIENIEM: Sprawdź konfigurację (PROJECT_ID, LOCATION, GCS_BUCKET, ENGINE_NAME) i uwierzytelnienie.")
    print(f"   Szczegóły błędu: {e}")

✅ Klienci Google Cloud zainicjalizowani pomyślnie.
--- 🏁 Rozpoczynam proces zapisu pamięci ---
➡️  Silnik agenta: projects/815755318672/locations/us-central1/reasoningEngines/6370486808450433024
➡️  Bucket GCS: memory_rag_for_agents

--- Przetwarzam rekord 1/3 | Misja: mission_20250823_215948_92ed8ebc ---
  ✅ [GCS] Zapisano obiekt: gs://memory_rag_for_agents/agent-memory-artifacts/mission_20250823_215948_92ed8ebc/d070a477f81b3401117a3fc3f409bdf4/final_plan.json


/home/jupyter/olga_zydziak/version_beta/advanced_agents_system/persist_missions_memory_semantic_incremental.py:57: ExperimentalWarning: The Vertex SDK GenAI agent engines module is experimental, and may change in future versions.
  vertex_client.agent_engines.create_memory(name=engine_name, fact=fact_json, scope=scope)


    ✅ [Vertex AI] Zapisano fakt 'mission_overview' (widok: overview)

--- Przetwarzam rekord 2/3 | Misja: mission_20250825_130540_a5cf18ab ---
  ✅ [GCS] Zapisano obiekt: gs://memory_rag_for_agents/agent-memory-artifacts/mission_20250825_130540_a5cf18ab/3d8be7f520a20960021c21d9509caedc/final_plan.json
    ✅ [Vertex AI] Zapisano fakt 'mission_overview' (widok: overview)

--- Przetwarzam rekord 3/3 | Misja: mission_20250825_132101_a5cf18ab ---
  ✅ [GCS] Zapisano obiekt: gs://memory_rag_for_agents/agent-memory-artifacts/mission_20250825_132101_a5cf18ab/6235fcace75c9874744d8b4f54808341/final_plan.json
    ✅ [Vertex AI] Zapisano fakt 'mission_overview' (widok: overview)

--- ✅ Proces zapisu zakończony ---
📊 Statystyki: Przetworzono 3, Sukces 3, Porażki 0
